In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import re

In [42]:
train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

In [43]:
train1 = train.drop(['Descript', 'Resolution'], axis = 1)
train1.head()

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541


In [44]:
train1 = train1.query('Y != 90')

In [45]:
# Ad_dict = {'.*?ST':'ST', '.*?AV':'AV', '.*?DR':'DR', '.*?I-80':'I-80', '.*?CT':'CT', '.*?LN':'LN','.*?WY':'WY',
#             '.*?HY':'HY', '.*?PL':'PL,','.*?BL':'BL','.*?TR':'TR','.*?AL':'AL','.*?CR':'CR','.*?PARK':'PARK',
#             '.*?WAY':'WAY', '.*?PZ':'PZ', '.*?MAR':'MAR','.*?RD':'RD', '.*?WK':'WK'}
# train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)
# train1['Address'].value_counts()

In [46]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

# Ad_dict = {'.*?ST / .*?ST':0, '.*?AV / .*?AV':1, '.*?ST / .*?AV':2, '.*?AV / .*?ST':3, '.*?ST':4, '.*?AV':5}
# train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)
# train1['Address'].unique()

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)

# Ad_dict = {'.*?ST':'ST', '.*?AV':'AV', '.*?DR':'DR', '.*?I-80':'I-80', '.*?CT':'CT', '.*?LN':'LN','.*?WY':'WY',
#             '.*?HY':'HY', '.*?PL':'PL,','.*?BL':'BL','.*?TR':'TR','.*?AL':'AL','.*?CR':'CR','.*?PARK':'PARK',
#             '.*?WAY':'WAY', '.*?PZ':'PZ', '.*?MAR':'MAR','.*?RD':'RD', '.*?WK':'WK'}
# train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)
# train1['Address'].value_counts()

train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# # DayOfWeek...label, PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,37,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,6,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,6,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,6,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23


In [47]:
test.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [48]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,0,3,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23
1,1,3,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23
2,2,3,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23
3,3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23
4,4,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23


In [49]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test[cols]

In [50]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [53]:
from sklearn.ensemble import RandomForestClassifier as RFC

def objective(trial):
    criterion = "gini"
    max_depth = trial.suggest_int('max_depth', 3, 7)
    n_estimators = 100
    max_features = "log2"

    rf = RFC(max_depth = max_depth,
        n_estimators = n_estimators,
        max_features = max_features,
        criterion = criterion)

    score = cross_val_score(rf, X_train, y_train, cv=5, scoring="r2")
    r2_mean = score.mean()
    print(r2_mean)

    return r2_mean


In [54]:
import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=100)


[I 2023-02-14 15:45:35,661] A new study created in memory with name: no-name-14804b37-a391-45da-9e88-783be7f22a1b
[I 2023-02-14 15:55:09,527] Trial 0 finished with value: -0.3883711884984994 and parameters: {'max_depth': 9, 'max_features': 'log2'}. Best is trial 0 with value: -0.3883711884984994.


-0.3883711884984994


[I 2023-02-14 16:04:26,819] Trial 1 finished with value: -0.37209540736049856 and parameters: {'max_depth': 8, 'max_features': 'auto'}. Best is trial 0 with value: -0.3883711884984994.


-0.37209540736049856


[I 2023-02-14 16:09:56,009] Trial 2 finished with value: -0.07334169754724237 and parameters: {'max_depth': 4, 'max_features': 'log2'}. Best is trial 0 with value: -0.3883711884984994.


...

-0.3936454507118149


[I 2023-02-15 17:34:52,012] Trial 98 finished with value: -0.38395799803221103 and parameters: {'max_depth': 9, 'max_features': 'auto'}. Best is trial 34 with value: -0.3979148805040095.


-0.38395799803221103


[I 2023-02-15 17:40:53,665] Trial 99 finished with value: -0.39054326971501563 and parameters: {'max_depth': 10, 'max_features': 'auto'}. Best is trial 34 with value: -0.3979148805040095.


-0.39054326971501563


In [56]:
optimised_rf = RFC(criterion="gini", max_depth=study.best_params['max_depth'], n_estimators=100, max_features=study.best_params['max_features'])
optimised_rf.fit(X_train ,y_train)

RandomForestClassifier(max_depth=10, max_features='log2')

In [57]:

# rf = RFC(n_jobs=-1,          # 複数のCPUコアを使って並列に学習します。-1は最大値。
#           random_state=2525)  # 乱数のシードです。
# rf = RFC(max_depth = 'max_depth',
#         n_estimators = 'n_estimators',
#         max_features = 'max_features',
#         criterion = 'gini')

# rf.fit(X_train, y_train)

In [59]:
y_pred = optimised_rf.predict(X_val)

In [22]:
# y_val.reshape(-1,1)

In [23]:
# print(f"acc: {rf.score(y_val, y_pred)}")

In [61]:
accuracy_score(y_val, y_pred)

0.26987224510715846

In [62]:
rf_submission = optimised_rf.predict(X_test)

In [64]:
import csv
pred = optimised_rf.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [65]:
submission.to_csv("rf1-3.csv", index=False)